In [2]:
import os
import json
import cv2

# Define a mapping from category names to class indices.
# (Adjust these to match your dataset and desired classes.)
class_mapping = {
    "traffic sign": 0,
    "car": 1,
    "person": 2,
    # Add other categories if needed, e.g.:
    "area/drivable": 3,
    "area/alternative": 4,
    "lane/road curb": 5,
    "lane/single white": 6,
}

def convert_json_to_yolo(json_path, image_path, output_txt_path, class_mapping):
    """
    Converts a JSON annotation to YOLO text format.
    The YOLO format is: <class_index> <x_center_norm> <y_center_norm> <width_norm> <height_norm>
    """
    # Load the image to get its dimensions
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not read image {image_path}")
        return
    height, width = img.shape[:2]

    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Here we assume each JSON has a "frames" list.
    # We'll use the first frame's annotations.
    if not data.get("frames"):
        print(f"No frames found in {json_path}")
        return

    frame = data["frames"][0]
    lines = []
    for obj in frame.get("objects", []):
        category = obj.get("category")
        # We only process objects with a "box2d" key.
        if "box2d" in obj:
            box = obj["box2d"]
            x1 = box["x1"]
            y1 = box["y1"]
            x2 = box["x2"]
            y2 = box["y2"]

            # Compute center, width, and height
            x_center = (x1 + x2) / 2.0
            y_center = (y1 + y2) / 2.0
            box_width = x2 - x1
            box_height = y2 - y1

            # Normalize coordinates by image dimensions
            x_center_norm = x_center / width
            y_center_norm = y_center / height
            width_norm = box_width / width
            height_norm = box_height / height

            # Map the category to an integer label
            if category not in class_mapping:
                # Skip objects not in our mapping.
                continue
            class_idx = class_mapping[category]

            line = f"{class_idx} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}"
            lines.append(line)
    
    # Write to the output .txt file (YOLO format)
    with open(output_txt_path, 'w') as f:
        for line in lines:
            f.write(line + "\n")

def process_dataset(json_base, img_base):
    """
    Converts all JSON files in json_base to YOLO format and writes the output labels
    into the corresponding image directory (img_base).
    Assumes that both json_base and img_base have identical subdirectories: 'train', 'val', 'test'.
    """
    for subset in ['train', 'val', 'test']:
        json_dir = os.path.join(json_base, subset)
        img_dir = os.path.join(img_base, subset)
        # Process each JSON file in the JSON directory.
        for filename in os.listdir(json_dir):
            if filename.endswith(".json"):
                json_path = os.path.join(json_dir, filename)
                base_name = os.path.splitext(filename)[0]
                # Assume the corresponding image is a .jpg file in the image directory.
                image_path = os.path.join(img_dir, base_name + ".jpg")
                # The YOLO label file will be saved with the same name but a .txt extension.
                output_txt_path = os.path.join(img_dir, base_name + ".txt")
                convert_json_to_yolo(json_path, image_path, output_txt_path, class_mapping)
                # print(f"Converted {json_path} -> {output_txt_path}")

# Run the conversion for each subset.
process_dataset("100k_labels", "100k")